In [129]:
import requests, json, xmltodict
import openai

def get_cid(bvid, max_page=10):
    """
    Get the cid of a bilibili video from its bvid.
    """

    url = "https://api.bilibili.com/x/web-interface/view?bvid=" + str(bvid)
    response = requests.get(url)
    res_dict = json.loads(response.text)

    cookie = r"LIVE_BUVID=AUTO3715893391262763; balh_mode=replace; balh_server_inner=__custom__; balh_is_closed=; buvid3=6117E460-4D82-4B15-BE1A-A55C9FD10FC618537infoc; buvid4=9518E84A-AA46-DBB3-2C8F-8030D361F84D25428-022050514-ly5N23H31hRIBxdxwOmqvQ==; buvid_fp_plain=undefined; hit-dyn-v2=1; b_nut=100; CURRENT_FNVAL=4048; _uuid=3C447151-E92D-BF57-10B97-E53BB85910C10252753infoc; rpdid=|(kkuum|~uRm0J'uY~|~mY|Ru; hit-new-style-dyn=1; SESSDATA=82dff401,1686423352,82d10*c2; bili_jct=36ea9e991cf030b5126e1d1558ac4e67; DedeUserID=14708458; DedeUserID__ckMd5=1bd4979b8de02688; i-wanna-go-back=-1; b_ut=5; nostalgia_conf=-1; header_theme_version=CLOSE; fingerprint=b2847ca1783dc8f8baadf44ec9d10a20; buvid_fp=9eb5471c66f7cf12232d2dc31ff4ce47; home_feed_column=4; PVID=2; CURRENT_QUALITY=0; bp_video_offset_14708458=769898593419001900; innersign=0; b_lsid=2CF10ED4A_186B6419CC3"

    title    = res_dict['data']['title']
    subtitle = res_dict['data']['subtitle']

    headers = {
        'Cookie': cookie,
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)

    subtitle_url = None
    if len(data['data']['subtitle']['list']) > 0:
        subtitle_url = data['data']['subtitle']['list'][0]['subtitle_url']


    num_comments = data['data']['stat']['reply']
    comments = []

    num_pages = num_comments // 20 + 1
    for n in range(1, min(num_pages, max_page) + 1):
        url = "https://api.bilibili.com/x/v2/reply?jsonp=jsonp&pn="+str(n)+"&type=1&oid="+str(bvid)+"&sort=1&nohot=1"
        req = requests.get(url)
        text = req.text
        json_text_list = json.loads(text)

        if json_text_list["data"]["replies"] is not None:
            for i in json_text_list["data"]["replies"]:
                comments.append("%s: %s"%(i["member"]["uname"],i["content"]["message"]))

    subtitle = ""
    if subtitle_url is not None:
        subtitle_response = requests.get(subtitle_url, headers=headers)
        subtitle = ",".join([json.loads(subtitle_response.text)['body'][i]['content'] for i in range(len(json.loads(subtitle_response.text)['body']))])

    return title, subtitle, comments

def get_prompt_summarize_subtitle(title, subtitle):
    """
    Get the prompt for the given cid.
    """

    prompt = "我希望你是一名专业的视频内容编辑，帮我总结视频的内容精华。请你将视频字幕文本进行总结，然后以无序列表的方式返回，不要超过5条。记得不要重复句子，确保所有的句子都足够精简，清晰完整，祝你好运！"
    prompt = "我希望你是一名专业的视频内容编辑，你的工作是基于视频的字幕将视频改写成一篇文章。记得不要重复句子，确保所有的句子都足够精简，清晰完整，祝你好运！"
    prompt += "\n视频标题：" + title
    prompt += "\n视频字幕：" + subtitle

    return prompt

def get_prompt_summarize_comments(title, comments):
    """
    Get the prompt for the given cid.
    """

    prompt  = "我希望你是一名专业的视频内容编辑，总结评论中被大家认可的观点。请你将视频评论进行总结，然后以无序列表的方式返回，不要超过5条。记得不要重复句子，确保所有的句子都足够精简，清晰完整，祝你好运！"
    prompt += "\n视频标题：" + title
    prompt += "\n以下是视频评论，每条评论前有其编号："

    for i, comment in enumerate(comments):
        prompt += "\n" + str(i) + ". " + comment

    return prompt

t ,s, c = get_cid("BV1ij411g7hN", max_page=1)


Title: 【鬼谷闲谈】水稻驯化史：干饭人是怎么养成的


In [130]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user", 
    "content": get_prompt_summarize_subtitle(t, s)[:2000]
    },
  ],
  temperature = 0.5,
  top_p = 1,
  frequency_penalty = 0,
  presence_penalty = 0,
  max_tokens = 400,
  stream = False, n = 1,
)
print(completion["choices"][0]["message"]["content"])

了几千年的驯化和改良后，水稻成为了中国人最重要的农作物之一。野生的水稻和其他禾本科植物一样，虽然能够在旱涝中生长，但无法在水中生根发芽。然而，祖先们演化出了一种特殊的基因，让水稻可以在没有氧气时通过糖酵起来产生能量，从而催动嫩芽快速生长，冲到水面呼吸空气。这让水稻能够在潜水沼泽中生长茁壮。虽然在距今约1万年前，当地人已经开始食用野生稻做的大米饭，但直到距今约8000年前，中国人才正式开始驯化水稻，并在约8000年前大体完成了对水稻的驯化。在驯化和改良过程中，水稻逐渐成型，爆杆挺直，颜色也从红色变成了米色。此外，南方人驯养的水牛也正好能用来耕种泥泥泞的水田，使得水稻成为了中国人最重要的农作物之一。


In [131]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user", 
    "content": get_prompt_summarize_comments(t, c)[:2000]
    },
  ],
  temperature = 0.5,
  top_p = 1,
  frequency_penalty = 0,
  presence_penalty = 0,
  max_tokens = 400,
  stream = False, n = 1,
)
print(completion["choices"][0]["message"]["content"])

1. 世界四大粮商控制麦子价格，但东亚东南亚以稻米为主食，稻米产量高于麦子，稻米自给率高。
2. 埃及是世界上水稻亩产最高的国家，尽管粮食自给率只有40%多，但凭借河流与肥沃的泥土，埃及的农作物单产大多都只能用令人发指来形容。
3. 菰是菰黑粉菌侵入后根茎膨大的产物，本来是种病害，但被发现很好吃，逐渐从菰米衍生成茭白。
4. 野生雄性不育株对于水稻育种非常重要，相当于灾民突然拿到了免费自助餐厅的钥匙。
5. 水稻是国内科研人员一步一步研究出来的，国内营销号却还在造谣水稻，却不知道科技与狠活的重要性。

